In [1]:
import requests
from scrapy.http import TextResponse
import pandas as pd
import time

Problem 1

In [2]:
def one_page_scraper(response):
    base_url = 'http://quotes.toscrape.com'
    quotes = response.css(".quote .text::text").extract()
    authors = response.css("small.author::text").extract()
    tags = [i.css(".tag::text").extract() for i in response.css(".quote .tags")]
    links = [ base_url+i for i in response.css("small.author ~ a::attr(href)").extract()]
    
    return {"Quote":quotes,"Author":authors,"Tag":tags,"Link":links};

In [3]:
def quotes_scraper(URL):
    base_url = 'http://quotes.toscrape.com';
    data = []
    finished = False
    while(finished == False):
        page = requests.get(URL);
        response = TextResponse(body=page.text,url=URL,encoding="utf-8")
        data.append(pd.DataFrame(one_page_scraper(response)))
        if len(response.css(".next a::attr(href)")) > 0:
            URL = base_url + response.css(".next a::attr(href)").extract()[0]
        else:
            finished = True
        time.sleep(1)
            
    return pd.concat(data)

In [4]:
quotes_scraper("http://quotes.toscrape.com/")

,Quote,Author,Tag,Link
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]",http://quotes.toscrape.com/author/Albert-Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]",http://quotes.toscrape.com/author/J-K-Rowling
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]",http://quotes.toscrape.com/author/Albert-Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]",http://quotes.toscrape.com/author/Jane-Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]",http://quotes.toscrape.com/author/Marilyn-Monroe
...,...,...,...,...
5,“You never really understand a person until yo...,Harper Lee,[better-life-empathy],http://quotes.toscrape.com/author/Harper-Lee
6,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,...",http://quotes.toscrape.com/author/Madeleine-LE...
7,“Never tell the truth to people who are not wo...,Mark Twain,[truth],http://quotes.toscrape.com/author/Mark-Twain
8,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational],http://quotes.toscrape.com/author/Dr-Seuss


Problem 2

In [5]:
def movie_scraper(URL):
    page = requests.get(URL)
    response = TextResponse(body=page.text,url=URL,encoding="utf-8")
    titles = response.css(".titleColumn a::text").extract()
    years = [i[1:5] for i in response.css(".titleColumn > .secondaryInfo::text").extract()]
    rankings = [i.css("::text").extract()[0][:-1].replace("\n(no change)","") for i in response.css(".velocity")]
    ratings = []
    for i in response.css(".imdbRating"):
        rating =  i.css("::text").extract()
        if len(rating) == 3:
            ratings.append(rating[1])
        else:
            ratings.append('')
    base_url = "https://www.imdb.com"
    links= [base_url+i for i in response.css(".titleColumn a::attr(href)").extract()]

    return pd.DataFrame({'Title':titles,"Year":years,"Ranking":rankings,"Rating":ratings,"Link":links})

In [6]:
url = 'https://www.imdb.com/chart/moviemeter/'

In [7]:
movie_scraper(url)

,Title,Year,Ranking,Rating,Link
0,365 dni,2020,1,3.5,https://www.imdb.com/title/tt10886166/
1,Da 5 Bloods,2020,2,6.7,https://www.imdb.com/title/tt9777644/
2,Artemis Fowl,2020,3,4.1,https://www.imdb.com/title/tt3089630/
3,The King of Staten Island,2020,4,7.2,https://www.imdb.com/title/tt9686708/
4,Knives Out,2019,5,7.9,https://www.imdb.com/title/tt8946378/
...,...,...,...,...,...
95,The Rising Hawk,2019,96,6.2,https://www.imdb.com/title/tt7439064/
96,Bloodshot,2020,97,5.7,https://www.imdb.com/title/tt1634106/
97,The Room,2003,98,3.7,https://www.imdb.com/title/tt0368226/
98,Emma.,2020,99,6.8,https://www.imdb.com/title/tt9214832/


Problem 3

In [8]:
def one_page_scraper(response):
    base_url = 'http://books.toscrape.com/catalogue/';
    pic_base_url = "http://books.toscrape.com"
    titles = response.css(".product_pod h3 a::attr(title)").extract()
    ratings = [i.replace("star-rating ","") for i in response.css(".star-rating::attr(class)").extract()]
    prices = [i.replace("Â£","") for i in response.css(".product_price .price_color::text").extract()]
    book_links = [base_url+i for i in response.css(".product_pod h3 a::attr(href)").extract()]
    picture_links = [i.replace("..", pic_base_url) for i in response.css(".image_container a img::attr(src)").extract()]
    in_stock = [i.replace("\n","").strip() for i in response.css(".availability::text").extract() if i!='\n    ']
   
    return {"Title":titles,"Rating": ratings,"Price":prices, "Book Link": book_links, "Picture Link": picture_links,"Availability": in_stock};

In [9]:
def books_scraper(URL):
    base_url = 'http://books.toscrape.com/catalogue/';
    data = []
    finished = False
    while(finished == False):
        page = requests.get(URL);
        response = TextResponse(body=page.text,url=URL,encoding="utf-8")
        data.append(pd.DataFrame(one_page_scraper(response)))
        time.sleep(1)
        if len(response.css(".next a::attr(href)")) > 0:
            URL = base_url + response.css(".next a::attr(href)").extract()[0]
        else:
            finished = True
      
    return pd.concat(data)

In [10]:
books_scraper("http://books.toscrape.com/catalogue/page-1.html")

,Title,Rating,Price,Book Link,Picture Link,Availability
0,A Light in the Attic,Three,51.77,http://books.toscrape.com/catalogue/a-light-in...,http://books.toscrape.com/media/cache/2c/da/2c...,In stock
1,Tipping the Velvet,One,53.74,http://books.toscrape.com/catalogue/tipping-th...,http://books.toscrape.com/media/cache/26/0c/26...,In stock
2,Soumission,One,50.10,http://books.toscrape.com/catalogue/soumission...,http://books.toscrape.com/media/cache/3e/ef/3e...,In stock
3,Sharp Objects,Four,47.82,http://books.toscrape.com/catalogue/sharp-obje...,http://books.toscrape.com/media/cache/32/51/32...,In stock
4,Sapiens: A Brief History of Humankind,Five,54.23,http://books.toscrape.com/catalogue/sapiens-a-...,http://books.toscrape.com/media/cache/be/a5/be...,In stock
...,...,...,...,...,...,...
15,Alice in Wonderland (Alice's Adventures in Won...,One,55.53,http://books.toscrape.com/catalogue/alice-in-w...,http://books.toscrape.com/media/cache/96/ee/96...,In stock
16,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Four,57.06,http://books.toscrape.com/catalogue/ajin-demi-...,http://books.toscrape.com/media/cache/09/7c/09...,In stock
17,A Spy's Devotion (The Regency Spies of London #1),Five,16.97,http://books.toscrape.com/catalogue/a-spys-dev...,http://books.toscrape.com/media/cache/1b/5f/1b...,In stock
18,1st to Die (Women's Murder Club #1),One,53.98,http://books.toscrape.com/catalogue/1st-to-die...,http://books.toscrape.com/media/cache/2b/41/2b...,In stock
